In [6]:
import os
from my_tools import *
from dependency import *
from instrumentation import *
from insert_patch import *
from program_synthesis import *
from get_heap_object import *
import argparse

In [ ]:
parser = argparse.ArgumentParser(description='file name')
parser.add_argument('f',type=str, help='file name')
args = parser.parse_args()
filename=args.f

In [ ]:
fuzz_cmd=[
    r'gcc -fpreprocessed -dD -E src.c | sed "/#/c\ " | sed "1d"',
    r"clang-12 -g -fsanitize=address,fuzzer src.c",
    r"./a.out -max_total_time=5 -keep_seed=1 -max_len=2 2>fuzzer_result"]



In [ ]:
def main(filename):
    os.system(f"gcc -fpreprocessed -dD -E {filename} | sed \"1d\" > {filename}")
    os.system(f"clang-12 -g -fsanitize=address,fuzzer {filename}")
    os.system(r"./a.out -max_total_time=5 -keep_seed=1 -max_len=2 2>fuzzer_result")
    os.system(f"cat {filename} | sed \"/#/c\ \" > dep_{filename} ")
    
    dep = get_dependency("fuzzer_result")
    inst_filename = instrument(dep)
    
    os.system(f"clang-12 -g -fsanitize=address,fuzzer {inst_filename}")
    os.system(r"./a.out -max_total_time=5 -keep_seed=1 -max_len=2 2>inst_result")
    
    syn_inf = get_synthesis_inf(dep,"inst_result")
    # run 100 times to get more dynamic value
    for i in range(100):
        os.system(r"./a.out -max_total_time=5 -keep_seed=1 -max_len=2 2>inst_result")
        add_dynamic_value(syn_inf,"inst_result")
        
    for err_inf,err_dep in zip(syn_inf,dep):
        patch_cand = synthesis().synthesis(err)
        filelist = read_file(filename)
        filelist,patch = insert_tmp_var(filelist,patch_cand)
        for k,v in patch.items():
            o,otype,line=get_error_object(err_dep,k,v)
        filelist,o=insert_heap_object(filelist,o,otype,line)
        free=cur_patch(patch,o)
        
        insert_patch(filelist,line,free)

In [ ]:
file=open("inst_result",'r')
inf=file.read()
for i in inf.split("\n"):
    if "instrument:" in i :
        print(i)

In [10]:
syn_inf=get_synthesis_inf(dep,"inst_result")

In [11]:
syn_inf

[{'error': [0, 1],
  'var': {'func': [{'state': 'input',
     'type': 'int',
     'name': 'a',
     'coord': 'dep__src.c:5:14',
     'value': [0, 5],
     'ret': [{'coord': 'dep__src.c:10:9'},
      {'coord': 'dep__src.c:12:9'},
      {'coord': 'dep__src.c:12:16'}]}]}}]

In [ ]:
add_dynamic_value(syn_inf,"inst_result")

In [ ]:
syn_inf

In [ ]:
def add_dynamic_value(syn_inf,filename):
    inst = get_dynamic_value(filename)
    for v in inst.values(): 
        l=len(v)-1
        break
    for d in syn_inf:
        d["error"]+=[0]*(l-1)+[1]
        for func,v in d["var"].items():
            for var in v:
                key=(var["name"],var["coord"].split(":")[1])
                if key not in inst:
                    continue
                var["value"]+=inst[key][:-1]
    

In [ ]:
def get_dynamic_value(filename):
    file=open(filename,'r')
    inf=file.read()
    inst=[]
    for i in inf.split("\n"):
        if "instrument:" in i :
            inst.append(i)
    value={}
    for v in inst:
        v=v.split()
        key=(v[4],v[3][:-1])
        if key in value:
            value[key].append(int(v[-1]))
        else:
            value[key]=[int(v[-1])]
    return value

In [ ]:
get_dynamic_value("inst_result")

In [ ]:
def get_error_object(dep):
    for i in dep.values():
        filename=i["ret"][0]["coord"].split(":")[0]
    file_dict = file_to_dict(filename)
    

In [9]:
dep = get_dependency("fuzzer_result")

In [12]:
dep

[{'func': {'dep': [{'name': 'a',
     'state': 'input',
     'type': 'int',
     'coord': 'dep__src.c:5:14'},
    {'name': 'b', 'state': 'var', 'type': 'int*', 'coord': 'dep__src.c:7:13'}],
   'ret': [{'coord': 'dep__src.c:10:9'},
    {'coord': 'dep__src.c:12:9'},
    {'coord': 'dep__src.c:12:16'}],
   'object': ('p', 'int*', 6)}}]

In [ ]:
def get_synthesis_inf(dep,filename):
    inst = get_dynamic_value(filename)
    syn_inf=[]
    for v in inst.values(): 
        l=len(v)-1
        break
    for d in dep:
        tmp={"error":[0]*(l-1)+[1],"var":{}}
        cur_tmp=tmp["var"]
        for func,v in d.items():
            ret=v["ret"]
            cur_tmp[func]=[]
            for var in v["dep"]:
                key=(var["name"],var["coord"].split(":")[1])
                if key not in inst:
                    continue
                cur_tmp[func].append({
                    "state":var["state"],
                    "type":var["type"],
                    "name":var["name"],
                    "coord":var["coord"],
                    "value":inst[key][:-1],
                    "ret":ret
                })
        syn_inf.append(tmp)
    return syn_inf
             
             
             
             
             

In [ ]:
data={
        "error":[1,1,1,1,1,1,1,1,1,1],
        "var":{"func":[
            {
                "state":"input",
                "type":"int",
                "name":"a",
                "coord":2,
                "value":[2,2,6,6,6,5,4,3,4,10],
                "ret":[]
            },
            {
                "state":"input",
                "type":"int",
                "name":"b",
                "coord":2,
                "value":[0,1,-2,-1,0,2,4,6,6,1],
                "ret":[]
            },
            {
                "state":"var",
                "type":"int",
                "name":"c",
                "coord":6,
                "value":[2,3,4,5,6,7,8,9,10,11],
                "ret":[]
            }
        ]}
    
}